In [1]:
from gliner import GLiNER
import pandas as pd

c:\Users\abhin\miniconda3\envs\nlpEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")
model.eval()

c:\Users\abhin\miniconda3\envs\nlpEnv\lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


GLiNER(
  (token_rep_layer): TokenRepLayer(
    (bert_layer): TransformerWordEmbeddings(
      (model): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128004, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-11): 12 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): Linear(in_features=768, out_features=768, bias=True)
                  (key_proj): Linear(in_features=768, out_features=768, bias=True)
                  (value_proj): Linear(in_features=768, out_features=768, bias=True)
                  (pos_dropout): StableDropout()
                  (dropout): StableDropout()
                )
                (output): DebertaV2SelfOutput(
                  (dense): Linear(in_featu

In [4]:
def get_objects(text):
    labels = ["ORGANIZATION"]
    entities = model.predict_entities(text, labels, threshold=0.4)
    my_dict = {}
    # for ent in entities:
    #     my_dict.setdefault(f"spacy_default_{ent.label_}", []).append(ent.text)

    for ent in entities:
        my_dict.setdefault(f"spacy_gliner_{ent['label']}", []).append(ent["text"])
    
    return pd.Series(my_dict)

In [5]:
df = pd.read_csv("b1.csv")
df["Sentences"] = df.Sentences.apply(lambda x:x.lstrip().replace(";", ","))
df = pd.concat([df, df.Sentences.apply(get_objects)], axis=1)
df

,Sentences,spacy_gliner_ORGANIZATION
0,"I love my new Nike sneakers, they're so comfor...",[Nike]
1,McDonald's is my go-to place for a quick burge...,[McDonald's]
2,"I need to buy some new Apple products, their l...",[Apple]
3,"Coca-Cola is my favorite soft drink, nothing b...",[Coca-Cola]
4,"I always buy my groceries from Walmart, they h...",[Walmart]
...,...,...
95,"I love the taste of Blue Bell frozen yogurt, i...",[Blue Bell]
96,"I always buy my groceries from Safeway, they h...",[Safeway]
97,I recently switched to using Gain laundry dete...,[Gain]
98,"I'm a loyal customer of Disney+, I love watchi...",[Disney+]


In [6]:
df[df.spacy_gliner_ORGANIZATION.isna()]

,Sentences,spacy_gliner_ORGANIZATION
45,I always drink Red Bull when I need a pick-me-...,NaN
55,"I love the new flavors of Lay's Stax chips, th...",NaN
64,I always drink Monster energy drinks when I ne...,NaN
89,I love the new flavors of Lay's Poppables chip...,NaN
93,"I always use Ivory soap, it's gentle on my skin",NaN
